In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import datetime as dt
from statsmodels.tsa.seasonal import seasonal_decompose
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
class DemandForecastingAgent:
    """
    Agent responsible for predicting future demand based on historical data,
    seasonal patterns, and external factors.
    """
    
    def __init__(self):
        self.model = RandomForestRegressor(n_estimators=100, random_state=42)
        self.scaler = StandardScaler()
        
    def preprocess_data(self, data):
        """Process historical sales data"""
        # Extract time-based features
        data['date'] = pd.to_datetime(data['date'])
        data['day_of_week'] = data['date'].dt.dayofweek
        data['month'] = data['date'].dt.month
        data['day'] = data['date'].dt.day
        data['quarter'] = data['date'].dt.quarter
        
        # One-hot encode categorical variables
        if 'product_category' in data.columns:
            data = pd.get_dummies(data, columns=['product_category'], drop_first=True)
        if 'store_id' in data.columns:
            data = pd.get_dummies(data, columns=['store_id'], drop_first=True)
            
        return data
    
    def train(self, historical_data):
        """Train the demand forecasting model"""
        data = self.preprocess_data(historical_data.copy())
        
        # Features and target
        X = data.drop(['date', 'sales'], axis=1)
        y = data['sales']
        
        # Scale features
        X_scaled = self.scaler.fit_transform(X)
        
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
        
        # Train model
        self.model.fit(X_train, y_train)
        
        # Evaluate
        y_pred = self.model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        
        print(f"Model trained with MSE: {mse:.2f}, MAE: {mae:.2f}")
        
        return {'mse': mse, 'mae': mae}
    
    def predict_demand(self, future_data):
        """Predict future demand"""
        data = self.preprocess_data(future_data.copy())
        
        # Drop target column if exists
        if 'sales' in data.columns:
            data = data.drop('sales', axis=1)
            
        # Extract features
        X = data.drop(['date'], axis=1)
        X_scaled = self.scaler.transform(X)
        
        # Make predictions
        predictions = self.model.predict(X_scaled)
        
        return predictions


class InventoryOptimizationAgent:
    """
    Agent responsible for monitoring inventory levels and determining
    optimal stock levels based on demand forecasts and constraints.
    """
    
    def __init__(self, holding_cost_percent=0.25, stockout_cost_factor=1.5):
        self.holding_cost_percent = holding_cost_percent  # Annual holding cost as percent of item value
        self.stockout_cost_factor = stockout_cost_factor  # Cost factor for stockouts
        
    def calculate_safety_stock(self, demand_mean, demand_std, lead_time, service_level=0.95):
        """Calculate safety stock based on demand variability and service level"""
        import scipy.stats as stats
        
        # Service level factor (z-score)
        z = stats.norm.ppf(service_level)
        
        # Safety stock calculation
        safety_stock = z * demand_std * np.sqrt(lead_time)
        
        return safety_stock
    
    def calculate_reorder_point(self, demand_mean, demand_std, lead_time, service_level=0.95):
        """Calculate reorder point"""
        safety_stock = self.calculate_safety_stock(demand_mean, demand_std, lead_time, service_level)
        reorder_point = (demand_mean * lead_time) + safety_stock
        
        return reorder_point
    
    def calculate_economic_order_quantity(self, annual_demand, order_cost, item_cost):
        """Calculate EOQ using the Wilson formula"""
        daily_holding_cost = (item_cost * self.holding_cost_percent) / 365
        eoq = np.sqrt((2 * annual_demand * order_cost) / daily_holding_cost)
        
        return eoq
    
    def optimize_inventory_levels(self, items_data, demand_forecast):
        """
        Optimize inventory levels based on forecasted demand
        
        Parameters:
        - items_data: DataFrame with item_id, current_stock, cost, lead_time
        - demand_forecast: DataFrame with item_id and forecasted_demand
        
        Returns:
        - DataFrame with optimal stock levels and reorder points
        """
        results = []
        
        for _, item in items_data.iterrows():
            item_id = item['item_id']
            current_stock = item['current_stock']
            item_cost = item['cost']
            lead_time = item['lead_time']
            
            # Get forecasted demand for this item
            item_forecast = demand_forecast[demand_forecast['item_id'] == item_id]
            
            if len(item_forecast) > 0:
                # Calculate demand statistics
                demand_mean = item_forecast['forecasted_demand'].mean()
                demand_std = item_forecast['forecasted_demand'].std() if len(item_forecast) > 1 else demand_mean * 0.2
                annual_demand = demand_mean * 365
                
                # Calculate optimal inventory parameters
                safety_stock = self.calculate_safety_stock(demand_mean, demand_std, lead_time)
                reorder_point = self.calculate_reorder_point(demand_mean, demand_std, lead_time)
                eoq = self.calculate_economic_order_quantity(annual_demand, order_cost=50, item_cost=item_cost)
                
                # Determine if reorder is needed
                reorder_needed = current_stock <= reorder_point
                reorder_quantity = eoq if reorder_needed else 0
                
                results.append({
                    'item_id': item_id,
                    'current_stock': current_stock,
                    'optimal_safety_stock': safety_stock,
                    'reorder_point': reorder_point,
                    'eoq': eoq,
                    'reorder_needed': reorder_needed,
                    'reorder_quantity': reorder_quantity
                })
                
        return pd.DataFrame(results)


class SupplyChainCoordinationAgent:
    """
    Agent responsible for coordinating inventory allocation across locations
    and managing supplier relationships.
    """
    
    def __init__(self):
        self.stores = {}
        self.warehouses = {}
        self.suppliers = {}
        
    def register_location(self, location_id, location_type, capacity, location_data=None):
        """Register a location (store, warehouse, supplier)"""
        location = {
            'id': location_id,
            'type': location_type,
            'capacity': capacity,
            'inventory': {},
            'data': location_data or {}
        }
        
        if location_type == 'store':
            self.stores[location_id] = location
        elif location_type == 'warehouse':
            self.warehouses[location_id] = location
        elif location_type == 'supplier':
            self.suppliers[location_id] = location
    
    def update_inventory(self, location_id, item_id, quantity):
        """Update inventory for a specific location and item"""
        location_type = None
        location = None
        
        if location_id in self.stores:
            location_type = 'store'
            location = self.stores[location_id]
        elif location_id in self.warehouses:
            location_type = 'warehouse'
            location = self.warehouses[location_id]
        elif location_id in self.suppliers:
            location_type = 'supplier'
            location = self.suppliers[location_id]
            
        if location:
            if item_id in location['inventory']:
                location['inventory'][item_id] += quantity
            else:
                location['inventory'][item_id] = quantity
                
            print(f"Updated {location_type} {location_id} inventory for item {item_id}: {location['inventory'][item_id]}")
            return True
        else:
            print(f"Location {location_id} not found")
            return False
    
    def check_inventory(self, location_id, item_id):
        """Check inventory level for a specific location and item"""
        if location_id in self.stores:
            location = self.stores[location_id]
        elif location_id in self.warehouses:
            location = self.warehouses[location_id]
        elif location_id in self.suppliers:
            location = self.suppliers[location_id]
        else:
            return None
            
        return location['inventory'].get(item_id, 0)
    
    def allocate_inventory(self, reallocation_requests):
        """
        Allocate inventory based on optimization requests
        
        Parameters:
        - reallocation_requests: list of dicts with:
          {from_location, to_location, item_id, quantity}
        
        Returns:
        - List of successful transfers
        """
        successful_transfers = []
        
        for request in reallocation_requests:
            from_id = request['from_location']
            to_id = request['to_location']
            item_id = request['item_id']
            quantity = request['quantity']
            
            # Check if source has enough inventory
            source_qty = self.check_inventory(from_id, item_id)
            
            if source_qty >= quantity:
                # Update source and destination inventory
                self.update_inventory(from_id, item_id, -quantity)
                self.update_inventory(to_id, item_id, quantity)
                
                successful_transfers.append({
                    'from_location': from_id,
                    'to_location': to_id,
                    'item_id': item_id,
                    'quantity': quantity,
                    'timestamp': dt.datetime.now()
                })
                
                print(f"Transferred {quantity} units of item {item_id} from {from_id} to {to_id}")
            else:
                print(f"Transfer failed: Not enough inventory of item {item_id} at location {from_id}")
                
        return successful_transfers
    
    def generate_reallocation_plan(self, inventory_status, demand_forecast):
        """
        Generate a plan to reallocate inventory based on forecasted demand
        
        Parameters:
        - inventory_status: DataFrame with location_id, item_id, current_stock
        - demand_forecast: DataFrame with location_id, item_id, forecasted_demand
        
        Returns:
        - List of reallocation requests
        """
        reallocation_requests = []
        
        # Group by item to consider all locations together
        for item_id in inventory_status['item_id'].unique():
            item_inventory = inventory_status[inventory_status['item_id'] == item_id]
            item_forecast = demand_forecast[demand_forecast['item_id'] == item_id]
            
            # Join inventory and forecast
            analysis = pd.merge(
                item_inventory, 
                item_forecast,
                on=['location_id', 'item_id'],
                how='left'
            )
            
            # Calculate projected surplus/deficit
            analysis['projected_balance'] = analysis['current_stock'] - analysis['forecasted_demand']
            
            # Identify locations with surplus and deficit
            surplus_locations = analysis[analysis['projected_balance'] > 0].sort_values('projected_balance', ascending=False)
            deficit_locations = analysis[analysis['projected_balance'] < 0].sort_values('projected_balance')
            
            # Generate reallocation requests
            for _, deficit in deficit_locations.iterrows():
                deficit_qty = abs(deficit['projected_balance'])
                deficit_location = deficit['location_id']
                
                # Find surplus locations to fulfill this deficit
                remaining_deficit = deficit_qty
                
                for _, surplus in surplus_locations.iterrows():
                    if remaining_deficit <= 0:
                        break
                        
                    surplus_location = surplus['location_id']
                    available_surplus = surplus['projected_balance']
                    
                    if available_surplus > 0:
                        # Calculate transfer quantity
                        transfer_qty = min(remaining_deficit, available_surplus)
                        
                        # Create reallocation request
                        if transfer_qty > 0:
                            reallocation_requests.append({
                                'from_location': surplus_location,
                                'to_location': deficit_location,
                                'item_id': item_id,
                                'quantity': transfer_qty
                            })
                            
                            # Update remaining deficit and surplus for next iterations
                            remaining_deficit -= transfer_qty
                            
                            # Update the surplus location's available surplus
                            surplus_locations.loc[surplus_locations['location_id'] == surplus_location, 'projected_balance'] -= transfer_qty
            
        return reallocation_requests


class PricingIntelligenceAgent:
    """
    Agent responsible for dynamic pricing based on inventory levels,
    demand forecasts, and market conditions.
    """
    
    def __init__(self, price_elasticity_data=None):
        self.price_elasticity = price_elasticity_data or {}
        
    def calculate_price_elasticity(self, historical_data):
        """Calculate price elasticity based on historical data"""
        elasticity_results = {}
        
        # Group by item
        for item_id in historical_data['item_id'].unique():
            item_data = historical_data[historical_data['item_id'] == item_id].sort_values('date')
            
            # Calculate percentage changes in price and quantity
            item_data['price_pct_change'] = item_data['price'].pct_change()
            item_data['sales_pct_change'] = item_data['sales'].pct_change()
            
            # Remove rows with NaN
            item_data = item_data.dropna()
            
            if len(item_data) > 1:
                # Calculate elasticity: % change in quantity / % change in price
                elasticity_values = item_data['sales_pct_change'] / item_data['price_pct_change']
                
                # Filter out extreme values
                valid_elasticity = elasticity_values[(elasticity_values > -10) & (elasticity_values < 10)]
                
                if len(valid_elasticity) > 0:
                    mean_elasticity = valid_elasticity.mean()
                    elasticity_results[item_id] = mean_elasticity
                    
        self.price_elasticity = elasticity_results
        return elasticity_results
    
    def recommend_price_adjustments(self, inventory_status, demand_forecast, current_prices, 
                                   min_price_adj=-0.15, max_price_adj=0.10):
        """
        Recommend price adjustments based on inventory status and demand forecast
        
        Parameters:
        - inventory_status: DataFrame with item_id, current_stock, weeks_supply
        - demand_forecast: DataFrame with item_id, forecasted_demand
        - current_prices: DataFrame with item_id, current_price
        - min_price_adj: minimum price adjustment (negative for price decrease)
        - max_price_adj: maximum price adjustment
        
        Returns:
        - DataFrame with recommended price adjustments
        """
        # Merge data
        merged_data = pd.merge(
            inventory_status,
            demand_forecast,
            on='item_id',
            how='left'
        )
        
        merged_data = pd.merge(
            merged_data,
            current_prices,
            on='item_id',
            how='left'
        )
        
        # Calculate weeks of supply
        merged_data['weeks_supply'] = merged_data['current_stock'] / merged_data['forecasted_demand']
        
        # Define ideal stock level range
        ideal_min_weeks = 4  # 4 weeks minimum stock
        ideal_max_weeks = 8  # 8 weeks maximum stock
        
        price_adjustments = []
        
        for _, item in merged_data.iterrows():
            item_id = item['item_id']
            current_price = item['current_price']
            weeks_supply = item['weeks_supply']
            
            # Get elasticity for this item (default to -1.5 if not available)
            elasticity = self.price_elasticity.get(item_id, -1.5)
            
            # Calculate adjustment factor based on weeks of supply
            if weeks_supply < ideal_min_weeks:
                # Low stock - increase price
                adj_factor = (ideal_min_weeks - weeks_supply) / ideal_min_weeks * max_price_adj
                adj_factor = min(adj_factor, max_price_adj)  # Cap at maximum
            elif weeks_supply > ideal_max_weeks:
                # High stock - decrease price
                adj_factor = (ideal_max_weeks - weeks_supply) / ideal_max_weeks * min_price_adj
                adj_factor = max(adj_factor, min_price_adj)  # Cap at minimum
            else:
                # Within ideal range - no change
                adj_factor = 0
                
            # Adjust factor based on elasticity
            if elasticity != 0:
                # If highly elastic (more negative), reduce the magnitude of price increases
                # If inelastic (closer to 0), price changes can be more aggressive
                elasticity_factor = min(1, max(0.2, 1 / abs(elasticity)))
                adj_factor *= elasticity_factor
                
            # Calculate new price
            new_price = current_price * (1 + adj_factor)
            
            price_adjustments.append({
                'item_id': item_id,
                'current_price': current_price,
                'weeks_supply': weeks_supply,
                'adjustment_factor': adj_factor,
                'new_price': new_price,
                'price_change_pct': adj_factor * 100
            })
            
        return pd.DataFrame(price_adjustments)


class IntelligentInventoryManagementSystem:
    """
    Main system that coordinates all agents to optimize inventory management
    """
    
    def __init__(self):
        self.demand_agent = DemandForecastingAgent()
        self.inventory_agent = InventoryOptimizationAgent()
        self.supply_chain_agent = SupplyChainCoordinationAgent()
        self.pricing_agent = PricingIntelligenceAgent()
        
        self.historical_data = None
        self.inventory_data = None
        self.location_data = None
        self.price_data = None
        
    def load_data(self, historical_data=None, inventory_data=None, location_data=None, price_data=None):
        """Load all required datasets"""
        self.historical_data = historical_data
        self.inventory_data = inventory_data
        self.location_data = location_data
        self.price_data = price_data
        
        # Register locations in supply chain agent
        if location_data is not None:
            for _, location in location_data.iterrows():
                self.supply_chain_agent.register_location(
                    location_id=location['location_id'],
                    location_type=location['location_type'],
                    capacity=location['capacity'],
                    location_data={k: location[k] for k in location.keys() if k not in ['location_id', 'location_type', 'capacity']}
                )
                
            # Load initial inventory into locations
            if inventory_data is not None:
                for _, inv in inventory_data.iterrows():
                    self.supply_chain_agent.update_inventory(
                        location_id=inv['location_id'],
                        item_id=inv['item_id'],
                        quantity=inv['current_stock']
                    )
        
        print("Data loaded successfully")
        
    def train_forecasting_model(self):
        """Train the demand forecasting model"""
        if self.historical_data is not None:
            print("Training demand forecasting model...")
            return self.demand_agent.train(self.historical_data)
        else:
            print("No historical data available for training")
            return None
        
    def generate_demand_forecast(self, future_dates):
        """Generate demand forecast for future dates"""
        print("Generating demand forecast...")
        
        # Create future data frame
        if self.historical_data is None:
            print("No historical data available for forecasting")
            return None
            
        # Get unique stores and products
        stores = self.historical_data['store_id'].unique() if 'store_id' in self.historical_data.columns else [1]
        products = self.historical_data['item_id'].unique() if 'item_id' in self.historical_data.columns else self.historical_data['product_id'].unique()
        
        # Create future dates for all combinations
        future_data = []
        for store in stores:
            for product in products:
                for date in future_dates:
                    future_data.append({
                        'store_id': store,
                        'item_id': product,
                        'date': date
                    })
                    
        future_df = pd.DataFrame(future_data)
        
        # Use some features from historical data
        if 'product_category' in self.historical_data.columns:
            product_categories = self.historical_data[['item_id', 'product_category']].drop_duplicates()
            future_df = pd.merge(future_df, product_categories, on='item_id', how='left')
            
        # Make predictions
        predictions = self.demand_agent.predict_demand(future_df)
        
        # Create forecast dataframe
        forecast_df = future_df.copy()
        forecast_df['forecasted_demand'] = predictions
        
        return forecast_df
    
    def optimize_inventory(self, demand_forecast):
        """Optimize inventory levels based on demand forecast"""
        print("Optimizing inventory levels...")
        
        if self.inventory_data is None:
            print("No inventory data available for optimization")
            return None
            
        # Calculate optimal inventory levels
        optimal_inventory = self.inventory_agent.optimize_inventory_levels(
            self.inventory_data,
            demand_forecast
        )
        
        return optimal_inventory
    
    def generate_reallocation_plan(self, demand_forecast):
        """Generate inventory reallocation plan"""
        print("Generating reallocation plan...")
        
        if self.inventory_data is None:
            print("No inventory data available for reallocation planning")
            return None
            
        # Prepare inventory status by location
        inventory_by_location = []
        for location_id in self.supply_chain_agent.stores:
            for item_id, qty in self.supply_chain_agent.stores[location_id]['inventory'].items():
                inventory_by_location.append({
                    'location_id': location_id,
                    'item_id': item_id,
                    'current_stock': qty
                })
                
        for location_id in self.supply_chain_agent.warehouses:
            for item_id, qty in self.supply_chain_agent.warehouses[location_id]['inventory'].items():
                inventory_by_location.append({
                    'location_id': location_id,
                    'item_id': item_id,
                    'current_stock': qty
                })
                
        inventory_status = pd.DataFrame(inventory_by_location)
        
        # Generate reallocation plan
        reallocation_plan = self.supply_chain_agent.generate_reallocation_plan(
            inventory_status,
            demand_forecast
        )
        
        return reallocation_plan
    
    def recommend_price_adjustments(self, demand_forecast):
        """Recommend price adjustments based on inventory and demand"""
        print("Recommending price adjustments...")
        
        if self.price_data is None or self.inventory_data is None:
            print("Missing price or inventory data for price recommendations")
            return None
            
        # Calculate price elasticity from historical data
        if self.historical_data is not None and 'price' in self.historical_data.columns:
            self.pricing_agent.calculate_price_elasticity(self.historical_data)
            
        # Get price recommendations
        price_recommendations = self.pricing_agent.recommend_price_adjustments(
            self.inventory_data,
            demand_forecast,
            self.price_data
        )
        
        return price_recommendations
    
    def run_full_optimization(self, forecast_days=30):
        """Run the complete optimization process"""
        print("Running full inventory optimization...")
        
        # 1. Generate future dates for forecasting
        today = dt.datetime.now().date()
        future_dates = [today + dt.timedelta(days=i) for i in range(1, forecast_days+1)]
        
        # 2. Train forecasting model
        self.train_forecasting_model()
        
        # 3. Generate demand forecast
        demand_forecast = self.generate_demand_forecast(future_dates)
        
        # 4. Optimize inventory levels
        optimal_inventory = self.optimize_inventory(demand_forecast)
        
        # 5. Generate reallocation plan
        reallocation_plan = self.generate_reallocation_plan(demand_forecast)
        
        # 6. Get price recommendations
        price_recommendations = self.recommend_price_adjustments(demand_forecast)
        
        # 7. Execute reallocation plan
        if reallocation_plan:
            successful_transfers = self.supply_chain_agent.allocate_inventory(reallocation_plan)
        else:
            successful_transfers = []
            
        # 8. Return comprehensive results
        results = {
            'demand_forecast': demand_forecast,
            'optimal_inventory': optimal_inventory,
            'reallocation_plan': reallocation_plan,
            'executed_transfers': successful_transfers,
            'price_recommendations': price_recommendations
        }
        
        return results
    
    def visualize_results(self, results):
        """Create visualizations of optimization results"""
        # Create figure with subplots
        fig, axes = plt.subplots(2, 2, figsize=(18, 12))
        
        # 1. Plot demand forecast
        if 'demand_forecast' in results and results['demand_forecast'] is not None:
            df = results['demand_forecast']
            forecast_by_date = df.groupby('date')['forecasted_demand'].sum().reset_index()
            axes[0, 0].plot(forecast_by_date['date'], forecast_by_date['forecasted_demand'], marker='o')
            axes[0, 0].set_title('Aggregated Demand Forecast')
            axes[0, 0].set_xlabel('Date')
            axes[0, 0].set_ylabel('Forecasted Demand')
            axes[0, 0].grid(True)
            
        # 2. Plot inventory optimization
        if 'optimal_inventory' in results and results['optimal_inventory'] is not None:
            df = results['optimal_inventory']
            df_sample = df.head(10)  # Limit to top 10 items for readability
            
            x = np.arange(len(df_sample))
            width = 0.35
            
            axes[0, 1].bar(x - width/2, df_sample['current_stock'], width, label='Current Stock')
            axes[0, 1].bar(x + width/2, df_sample['optimal_safety_stock'], width, label='Safety Stock')
            axes[0, 1].set_title('Current vs. Safety Stock Levels')
            axes[0, 1].set_xlabel('Item ID')
            axes[0, 1].set_ylabel('Stock Level')
            axes[0, 1].set_xticks(x)
            axes[0, 1].set_xticklabels(df_sample['item_id'])
            axes[0, 1].legend()
            
        # 3. Plot price recommendations
        if 'price_recommendations' in results and results['price_recommendations'] is not None:
            df = results['price_recommendations']
            df_sample = df.sort_values('price_change_pct').head(10)  # Top 10 price changes
            
            axes[1, 0].barh(df_sample['item_id'], df_sample['price_change_pct'])
            axes[1, 0].set_title('Recommended Price Changes')
            axes[1, 0].set_xlabel('Percent Change')
            axes[1, 0].set_ylabel('Item ID')
            axes[1, 0].axvline(x=0, color='k', linestyle='-', alpha=0.3)
            axes[1, 0].grid(True)
            
        # 4. Plot reallocation plan
        if ('reallocation_plan' in results and results['reallocation_plan'] and 
            len(results['reallocation_plan']) > 0):
            
            df = pd.DataFrame(results['reallocation_plan'])
            total_by_item = df.groupby('item_id')['quantity'].sum().reset_index()
            total_by_item = total_by_item.sort_values('quantity', ascending=False).head(10)
            
            axes[1, 1].bar(total_by_item['item_id'], total_by_item['quantity'])
            axes[1, 1].set_title('Top Items Being Reallocated')
            axes[1, 1].set_xlabel('Item ID')
            axes[1, 1].set_ylabel('Total Quantity')
            axes[1, 1].grid(True)
            
        plt.tight_layout()
        plt.show()
        
        return fig


# Example usage
if __name__ == "__main__":
    # Create sample data for demonstration
    # In a real scenario, this would come from your dataset
    
    # Sample historical sales data
    dates = pd.date_range(start='2023-01-01', end='2023-12-31')
    products = range(1, 11)  # 10 products
    stores = range(1, 6)     # 5 stores
    
    historical_rows = []
    for date in dates:
        for product in products:
            for store in stores:
                # Add some seasonality and trends
                base_demand = 50 + (product * 5) + (store * 2)
                day_of_week_effect = 1 + (0.2 if date.weekday() >= 5 else 0)  # Weekend boost
                month_effect = 1 + (0.3 if date.month in [11, 12] else 0)     # Holiday season boost
                
                # Add some randomness
                random_effect = np.random.normal(1, 0.1)
                
                sales = int(base_demand * day_of_week_effect * month_effect * random_effect)
                price = 10 + (product * 2) * (0.9 + np.random.normal(0, 0.05))
                
                historical_rows.append({
                    'date': date,
                    'store_id': store,
                    'item_id': product,
                    'sales': sales,
                    'price': price,
                    'product_category': f"Category_{(product-1)//3 + 1}"  # 4 categories
                })
    
    historical_data = pd.DataFrame(historical_rows

In [ ]:
historical_data = pd.DataFrame(historical_rows)
    
    # Sample inventory data
    inventory_rows = []
    for product in products:
        for store in stores:
            # Current stock with some variation
            base_stock = 100 + (product * 10)
            random_factor = np.random.normal(1, 0.3)
            current_stock = int(base_stock * random_factor)
            
            inventory_rows.append({
                'location_id': f"store_{store}",
                'item_id': product,
                'current_stock': current_stock,
                'cost': 5 + (product * 1.5),
                'lead_time': 3 + (product % 3)  # Lead time in days
            })
    
    # Add warehouse inventory
    for product in products:
        for warehouse_id in range(1, 3):  # 2 warehouses
            base_stock = 300 + (product * 20)
            random_factor = np.random.normal(1, 0.2)
            current_stock = int(base_stock * random_factor)
            
            inventory_rows.append({
                'location_id': f"warehouse_{warehouse_id}",
                'item_id': product,
                'current_stock': current_stock,
                'cost': 5 + (product * 1.5),
                'lead_time': 2 + (product % 2)  # Warehouses have slightly faster lead times
            })
    
    inventory_data = pd.DataFrame(inventory_rows)
    
    # Location data
    location_rows = []
    for store in stores:
        location_rows.append({
            'location_id': f"store_{store}",
            'location_type': 'store',
            'capacity': 2000,
            'region': f"Region_{(store-1)//2 + 1}"
        })
        
    for warehouse_id in range(1, 3):
        location_rows.append({
            'location_id': f"warehouse_{warehouse_id}",
            'location_type': 'warehouse',
            'capacity': 10000,
            'region': f"Region_{warehouse_id}"
        })
    
    location_data = pd.DataFrame(location_rows)
    
    # Price data
    price_rows = []
    for product in products:
        base_price = 15 + (product * 3)
        
        price_rows.append({
            'item_id': product,
            'current_price': base_price,
            'min_price': base_price * 0.8,
            'max_price': base_price * 1.2
        })
    
    price_data = pd.DataFrame(price_rows)
    
    # Create and run the system
    iims = IntelligentInventoryManagementSystem()
    iims.load_data(
        historical_data=historical_data,
        inventory_data=inventory_data,
        location_data=location_data,
        price_data=price_data
    )
    
    # Run full optimization
    results = iims.run_full_optimization(forecast_days=14)
    
    # Visualize results
    fig = iims.visualize_results(results)
    
    # Display detailed results
    print("\n===== OPTIMIZATION RESULTS =====")
    
    print("\n1. Demand Forecast Sample:")
    if results['demand_forecast'] is not None:
        print(results['demand_forecast'].head())
    
    print("\n2. Inventory Optimization Sample:")
    if results['optimal_inventory'] is not None:
        print(results['optimal_inventory'].head())
    
    print("\n3. Price Recommendations Sample:")
    if results['price_recommendations'] is not None:
        print(results['price_recommendations'].head())
    
    print("\n4. Reallocation Plan Sample:")
    if results['reallocation_plan']:
        for i, realloc in enumerate(results['reallocation_plan'][:5]):
            print(f"Transfer {i+1}: {realloc['quantity']} units of item {realloc['item_id']} "
                  f"from {realloc['from_location']} to {realloc['to_location']}")
    
    # Define a function to load real data
    def load_real_datasets(sales_file, inventory_file, location_file=None, price_file=None):
        """
        Load real datasets for the inventory management system
        
        Parameters:
        - sales_file: CSV file with historical sales data
        - inventory_file: CSV file with current inventory data
        - location_file: CSV file with location data
        - price_file: CSV file with pricing data
        
        Returns:
        - Tuple of DataFrames (sales_data, inventory_data, location_data, price_data)
        """
        # Load sales data
        sales_data = pd.read_csv(sales_file)
        
        # Ensure required columns exist
        required_sales_cols = ['date', 'item_id', 'sales']
        for col in required_sales_cols:
            if col not in sales_data.columns:
                print(f"Warning: Required column '{col}' not found in sales data")
        
        # Convert date to datetime
        if 'date' in sales_data.columns:
            sales_data['date'] = pd.to_datetime(sales_data['date'])
        
        # Load inventory data
        inventory_data = pd.read_csv(inventory_file)
        
        required_inv_cols = ['location_id', 'item_id', 'current_stock']
        for col in required_inv_cols:
            if col not in inventory_data.columns:
                print(f"Warning: Required column '{col}' not found in inventory data")
        
        # Load location data if provided
        location_data = None
        if location_file:
            location_data = pd.read_csv(location_file)
            
            required_loc_cols = ['location_id', 'location_type']
            for col in required_loc_cols:
                if col not in location_data.columns:
                    print(f"Warning: Required column '{col}' not found in location data")
        
        # Load price data if provided
        price_data = None
        if price_file:
            price_data = pd.read_csv(price_file)
            
            required_price_cols = ['item_id', 'current_price']
            for col in required_price_cols:
                if col not in price_data.columns:
                    print(f"Warning: Required column '{col}' not found in price data")
        
        return sales_data, inventory_data, location_data, price_data

    # Example to use with real data:
    # Load real datasets
    sales_data, inventory_data, location_data, price_data = load_real_datasets(
        'sales_history.csv',
        'current_inventory.csv',
        'locations.csv',
        'current_prices.csv'
    )
    
    # Create system with real data
    real_system = IntelligentInventoryManagementSystem()
    real_system.load_data(
        historical_data=sales_data,
        inventory_data=inventory_data,
        location_data=location_data,
        price_data=price_data
    )
    
    # Run optimization with real data
    real_results = real_system.run_full_optimization(forecast_days=30)
    real_system.visualize_results(real_results)